<a href="https://colab.research.google.com/github/ayoubbensakhria/finance_algo/blob/master/Product_Reviews_Scraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The goal is to scrap multiple reviews for many websites, summerize them with global note.

In [ ]:
# Setting-up Environment & Importing necessary libraries
!pip install selenium
!apt-get update
!apt install -y chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

from selenium import webdriver
from bs4 import BeautifulSoup
from shutil import make_archive
import pandas as pd

# Setting-up web driver
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver',options=options)

# **DATA SCRAPING**


In [ ]:
# base urls
amazon_url = "https://www.amazon.co.uk"
ebay_url = "https://www.ebay.co.uk"

# Product attr on the search page


In [5]:
base_url = "https://www.amazon.in"
dataframe = pd.DataFrame(columns=['product_title', 'customer_reviews_rate', 'offered_price','marked_price', 'is_prime_product', 'product_type', 'product_url'])

# Requesting keyword and Processing for search
keyword = input("Enter the search keyword: ")
keyword = keyword.replace(" ", "+")

# Requesting total page to be extracted for the keyword search
numbers_of_pages = 2
try:
  numbers_of_pages = int(input("Enter the number of pages to be extracted: "))
except:
  print("That's not a valid input! Value set to default")
  numbers_of_pages = 2

Enter the search keyword: how to crack product manager interview
Enter the number of pages to be extracted: 3


## *Assigning URL and fetching the source code for given page numbers.*

In [14]:
for page in range(1, numbers_of_pages + 1):
  try:
    search_url = base_url + "/s?k={}&page={}&ref=nb_sb_noss".format(keyword, page)
    driver.get(search_url)
    webpage = driver.execute_script("return document.body.outerHTML;")
    soup = BeautifulSoup(webpage, "html.parser")
    # Extracting the average customer reviews
    try:
      result_div = soup.findAll("div", attrs={"class": "s-main-slot s-result-list s-search-results sg-row"})
      for product_detail in result_div:
        product_title = ""
        product_url = ""
        customer_reviews_rate = ""
        offered_price = ""
        marked_price =""
        is_prime_product = ""
        product_type = ""

        # Fetching Product Title
        try:
          product_title = product_detail.find("span", attrs={"class": "a-size-medium a-color-base a-text-normal"}).text
        except:
          # Exception for no product title
          pass

        # Extracting the product URL
        try:
          partial_product_url = product_detail.find("a", attrs={"class": "a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal"}).attrs["href"]
          
        except:
          # Exception for no product url
          pass

        # Fetching Customer Rating, if available
        try:
          customer_reviews_rate = product_detail.find("div", attrs={"class": "a-row a-size-small"}).text
        except:
          # Exception for no previous reviews on the product
          customer_reviews_rate = "Not Available"
        
        # Fetching Offer Price
        try:
          offered_price = product_detail.find("span", attrs={"class": "a-price"}).find("span", attrs={"class": "a-offscreen"}).text
          if offered_price == '' : offered_price = "Not Available"
        except:
          # Exception for no offers
          offered_price = "Not Available"

        # Fetching Marked Price
        try:
          marked_price = product_detail.find("span", attrs={"class": "a-price a-text-price"}).find("span", attrs={"class": "a-offscreen"}).text
          if marked_price == '' : marked_price = "Not Available"
        except:
          # Exception for no offers
          marked_price = "Not Available"
        
        # Fetching Prime availability
        try:
          product_detail.find("span", attrs={"class": "aok-relative s-icon-text-medium s-prime"})
          is_prime_product = "Yes"
        except:
          is_prime_product = "No"
        
        # Fetching Product Type (sponsered or generic)
        try:
          product_detail.find("div", attrs={"class": "a-row a-spacing-micro"}).text
          product_type = "Sponsored"
        except:
          product_type = "Generic"
        
        
        # Appending DataFrame with new product entry
        dataframe = dataframe.append({
            'product_title': product_title,
            'customer_reviews_rate': customer_reviews_rate,
            'offered_price': offered_price,
            'marked_price': marked_price,
            'is_prime_product': is_prime_product,
            'product_type': product_type,
            'product_url': product_url
            }, ignore_index = True)
        
    except:
      # Exception for div find failure
      pass

  except:
    # Exception for No Results for search or page
    pass

/gp/slredirect/picassoRedirect.html/ref=pa_sp_atf_aps_sr_pg1_1?ie=UTF8&adId=A05089902SKP85YOCN8YT&url=%2FCracking-PM-Interview-Product-Technology%2Fdp%2F0984782818%2Fref%3Dsr_1_1_sspa%3Fkeywords%3Dhow%2Bto%2Bcrack%2Bproduct%2Bmanager%2Binterview%26qid%3D1654622763%26sr%3D8-1-spons%26psc%3D1&qualifier=1654622763&id=8862723641317980&widgetName=sp_atf
/gp/slredirect/picassoRedirect.html/ref=pa_sp_atf_next_aps_sr_pg2_1?ie=UTF8&adId=A04475512Q4IYJBS2DWOE&url=%2FFriends-Influence-People-Deluxe-Hardbound%2Fdp%2F9354403778%2Fref%3Dsr_1_17_sspa%3Fkeywords%3Dhow%2Bto%2Bcrack%2Bproduct%2Bmanager%2Binterview%26qid%3D1654622765%26sr%3D8-17-spons%26psc%3D1%26smid%3DA3H3WE9M6NY1KV&qualifier=1654622765&id=2304537914348767&widgetName=sp_atf_next
/gp/slredirect/picassoRedirect.html/ref=pa_sp_atf_next_aps_sr_pg3_1?ie=UTF8&adId=A014948022R6P7X5K4MRO&url=%2FGrip-working-smart-getting-matters%2Fdp%2F0008510571%2Fref%3Dsr_1_33_sspa%3Fkeywords%3Dhow%2Bto%2Bcrack%2Bproduct%2Bmanager%2Binterview%26qid%3D1654622

#Parse product URLs

In [16]:
dataframe.to_csv('amazon_product_details_by_keyword.csv')
dataframe

,product_title,customer_reviews_rate,offered_price,marked_price,is_prime_product,product_type,product_url
0,CareerCup Cracking The Pm Interview: How To La...,"4.5 out of 5 stars 1,729",₹530,"₹1,303",Yes,Sponsored,
1,How to Win Friends and Influence People (Delux...,"4.5 out of 5 stars 70,704",₹329,"₹1,099",Yes,Sponsored,
2,GRIP: The art of working smart (and getting to...,4.2 out of 5 stars 30,₹383,₹499,Yes,Sponsored,
3,CareerCup Cracking The Pm Interview: How To La...,"4.5 out of 5 stars 1,729",₹530,"₹1,303",Yes,Sponsored,
4,How to Win Friends and Influence People (Delux...,"4.5 out of 5 stars 70,704",₹329,"₹1,099",Yes,Sponsored,
5,GRIP: The art of working smart (and getting to...,4.2 out of 5 stars 30,₹383,₹499,Yes,Sponsored,
6,CareerCup Cracking The Pm Interview: How To La...,"4.5 out of 5 stars 1,729",₹530,"₹1,303",Yes,Sponsored,https://www.amazon.in/gp/slredirect/picassoRed...
7,How to Win Friends and Influence People (Delux...,"4.5 out of 5 stars 70,704",₹329,"₹1,099",Yes,Sponsored,https://www.amazon.in/gp/slredirect/picassoRed...
8,GRIP: The art of working smart (and getting to...,4.2 out of 5 stars 30,₹383,₹499,Yes,Sponsored,https://www.amazon.in/gp/slredirect/picassoRed...
